In [1]:
import stanza
import spacy_stanza
import dadmatools.pipeline.language as language
from dadmatools.models.normalizer import Normalizer
from parsivar import Normalizer as ParsivarNormalizer
import json 
import pytse_client as tse
import warnings
from spacy.language import Language
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
import pandas as pd
from tqdm import tqdm 
import spacy
from spacy.matcher import Matcher, PhraseMatcher
from spacy.tokens import Span
from spacy import displacy


2022-04-13 14:23:17.437193: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-13 14:23:17.437212: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Setup Stanza

In [2]:
stanza.install_corenlp()

2022-04-13 14:23:26 WARNING: Directory /home/ahura/stanza_corenlp already exists. Please install CoreNLP to a new directory.


2022-04-13 14:23:26,624 Directory /home/ahura/stanza_corenlp already exists. Please install CoreNLP to a new directory.


In [3]:
stanza.download('fa')

2022-04-13 14:23:27 INFO: Downloading default packages for language: fa (Persian)...


2022-04-13 14:23:27,817 Downloading default packages for language: fa (Persian)...


2022-04-13 14:23:30 INFO: File exists: /home/ahura/stanza_resources/fa/default.zip.


2022-04-13 14:23:30,417 File exists: /home/ahura/stanza_resources/fa/default.zip.


2022-04-13 14:23:34 INFO: Finished downloading models and saved to /home/ahura/stanza_resources.


2022-04-13 14:23:34,878 Finished downloading models and saved to /home/ahura/stanza_resources.


In [4]:
stanza_nlp = spacy_stanza.load_pipeline("fa")

2022-04-13 14:23:35 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |



2022-04-13 14:23:35,666 Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |



2022-04-13 14:23:35 INFO: Use device: gpu


2022-04-13 14:23:35,667 Use device: gpu


2022-04-13 14:23:35 INFO: Loading: tokenize


...

2022-04-13 14:23:39,935 Loading: lemma


2022-04-13 14:23:40 INFO: Loading: depparse


2022-04-13 14:23:40,031 Loading: depparse


2022-04-13 14:23:44 INFO: Done loading processors!


2022-04-13 14:23:44,119 Done loading processors!


# Setup DadmaTools

In [5]:
# # here lemmatizer and pos tagger will be loaded
# # as tokenizer is the default tool, it will be loaded as well even without calling
# pips = 'ner, pos, dep, cons, chunk, lem, tok' 
# dadma_nlp = language.Pipeline(pips)

# # you can see the pipeline with this code
# print(dadma_nlp.analyze_pipes(pretty=True))

# # dadma_doc is an SpaCy object

### Dadma Normalizer

In [6]:

normalizer1 = Normalizer(
    full_cleaning=False,
    unify_chars=True,
    refine_punc_spacing=True,
    remove_extra_space=True,
    remove_puncs=False,
    remove_html=False,
    remove_stop_word=False,
    replace_email_with="<EMAIL>",
    replace_number_with=None,
    replace_url_with="<URL",
    replace_mobile_number_with="<MOBILE_NUMBER>",
    replace_emoji_with="<EMOJI>",
    replace_home_number_with="<HOME_NUMBER>"
)

normalizer2 = ParsivarNormalizer()

# Test Libs

# Read & Write Symbols 

## Download And Write

In [7]:

# warnings.simplefilter(action='ignore', category=FutureWarning)

# symbol_to_price_records = tse.download(symbols="all", write_to_csv=True)

In [8]:
# symbols = list(symbol_to_price_records.keys())
# symbols = sorted(symbols, key=str.lower)


# with open('symbols.json', 'w',) as file:
#     json.dump({"symbols": symbols}, file, ensure_ascii=False)
    


In [9]:


# columns = ["Symbol", "Corp. Title", "TSE URL", "Group Name"]

# symbols_info_list = []
# for symbol in tqdm(x.keys()):
#     ticker = tse.Ticker(symbol, adjust=True, )

#     row = [symbol, ticker.title, ticker.url, ticker.group_name]

#     symbols_info_list.append(row)

# pd.DataFrame(symbols_info_list, )


## Write

In [10]:
with open('symbols.json', 'r') as file:
    symbols = json.load(file)['symbols']
    
len(symbols)

1201

# Sample Text

In [11]:
texts = [
    # "برکت همین افشای ب باعث می شود سهم سه درصد مثبت بشود. بخاطر همین میگم پیگیر باشید.",
    # "نماد برکت امروز عرضه اولیه خیلی خوبی داره."
    # 'روز چهارشنبه یک دفعه برای خودشون افشا زدن.'
    "سهام وغدیر و خزر کاهش یافت.",
    # "ریزش بازار به دلیل حمله‌ی روسیه هست.",
    # "فک کنم یه اصلاح قیمتی و کمی ریزش داشته باشیم.",
    # "قرارداد با آمریکا باعث افت قیمت سهم وغدیر شد",
    # "یک نکته‌ی تکنیکالی هم در صورت دستکاری نشدن اضافه کنم، کندلی که روز سه شنبه‌ی گذشته ثبت کرد کامل است",

    # "روز چهارشنبه یه دفعه برای خودشون افشا زدن",
    # "رشد قیمت‌ها باعث ایجاد صف خرید در سهم پرشیا شد",
    # "شاخص به ۲ میلیون می‌رسه",
    # "آمریکا باعث ریزش بازار شد",
    # "آمریکا موجب ریزش بازار شد",
    # "آمریکا دلیل ریزش بازار شد",
    # "کاهش قیمت سهم عجیب بود",
    # "قیمت زیاد شد",
    # "قیمت زیاد است",
    # "به کتابخانه رفتم.",
    # "به کتابخانه رفت.",
    # "پول در جیب من است.",
    # "سهم قیمتش پایین است",
    # "حضور تو موجب خوشحالی من در هوای بارانی است",
]

terms = [
    'ضرر',
    'سود',
    'اطلاعیه',
    'افزایش سرمایه',
    'تقسیم سود',
    'دامنه نوسان',
    'نوسان شدید',
    'سهم رانتی',
    'عرضه اولیه',
    'افشا',
    'فعالیت ماهانه',
    'فعالیت سالانه',
]

# Symbol Tagging Component

In [15]:
import sys
sys.setrecursionlimit(2000)

symbols_patterns = stanza_nlp.pipe(symbols, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])
matcher = PhraseMatcher(stanza_nlp.vocab)
matcher.add("SYMBOL", symbols_patterns)

@Language.component("symbol_component")
def symbol_component_function(doc): 
    
    matches = matcher(doc)
    
    spans = [Span(doc, start, end, label="SYMBOL") for match_id, start, end in matches]
    doc.ents = spans

    return doc

if "symbol_component" not in stanza_nlp.pipe_names:
    stanza_nlp.add_pipe("symbol_component", first=True)


['symbol_component']


(برکت,)

In [16]:
text = "نماد برکت امروز عرضه اولیه خیلی خوبی داره."
doc = stanza_nlp(text)
doc.ents


(برکت,)

### Stanza

## POS Matching 

In [13]:
nlp = stanza_nlp
matcher = Matcher(nlp.vocab, validate=True)

term_patterns = [[{"TEXT": {"REGEX": term + "*"}}] for term in terms]
symbol_patterns = [[{"TEXT": symbol}] for symbol in symbols]

matcher.add("TERM", term_patterns)
matcher.add("SYMBOL", symbol_patterns)

for t_index, text in enumerate(texts): 
    text = normalizer1.normalize(text)
    text = normalizer2.normalize(text)
    
    print(f"Normalized Text: {text}")
    doc = nlp(text)
    print(f'Text {t_index + 1}: ...')
    
    matches = matcher(doc)
    
    for m_index, (match_id, start, end) in enumerate(matches):
        span:Span = Span(doc, start, end, label=match_id)
        print(f"Match {m_index + 1}: {span.text}, {span.label_}")
        if span.label_ == "TERM":
            print(list(span.subtree))
        
        
    for s_index, sentence in enumerate(doc.sents):
        print(f'Sentence {s_index + 1}: ...')
    #     displacy.render(sentence, style="dep")
    #     print(f'\n\n')
                
    print("\n\n")

Normalized Text: سهام وغدیر و خزر کاهش یافت .
Text 1: ...
Match 1: وغدیر, SYMBOL
Match 2: خزر, SYMBOL
Sentence 1: ...





## Dependency Tree

In [14]:
# def children_string(children): 
#     return " ".join([ch.text for ch in children])
    
# # stanza_nlp.add_pipe("merge_noun_chunks")



# for t_index, text in enumerate(texts):
#     text = normalizer.normalize(text)
#     doc = stanza_nlp(text)
#     print(f'Text {t_index + 1}: ...')
        
#     for s_index, sentence in enumerate(doc.sents):
#         print(f'Sentence {s_index + 1}: ...')
#         # for token in sentence:
#             # print(f'word: {token.text:12}, pos: {token.pos_:10}, tag: {token.tag_:10}, dep: {token.dep_:15}')

#         print(f'\n\n')
#         print(f"sentence root: {sentence.root.text}")
#         for child1 in sentence.root.children:
#             print(f"{child1.text}, {child1.pos_}, {child1.tag_} {child1.dep_}")
            
            
        
#         displacy.render(sentence, style="dep")
        
                
#     print("\n\n")


NameError: name 'normalizer' is not defined

# Dadma

In [ ]:

# def children_string(children): 
#     return " ".join([ch.text for ch in children])

    
# for t_index, text in enumerate(texts):
#     text = normalizer.normalize(text)
#     doc = dadma_nlp(text)
#     print(f'Text {t_index + 1}: ...')
        
#     for s_index, sentence in enumerate(doc.sents):
#         print(f'Sentence {s_index + 1}: ...')
#         # for token in sentence:
#         #     print(f'word: {token.text:12}, pos: {token.pos_:10}, tag: {token.tag_:10}, dep: {token.dep_:15}')

#         print(f'\n\n')
#         print(f"sentence root: {sentence.root.text}")
#         # for child1 in sentence.root.children:
#             # print(f"{child1.text} Span is: {children_string(child1.subtree)}")
            
#         # print(f"constituency: {doc._.constituency}")
#         print(f"chunks: {doc._.chunks}")
        
    
                
#     print("\n\n")


# TestTest

## display Matchings

In [ ]:


# nlp = spacy.load("en_core_web_sm")
# matcher = Matcher(nlp.vocab)
# matched_sents = []  # Collect data of matched sentences to be visualized

# def collect_sents(matcher, doc, i, matches):
#     match_id, start, end = matches[i]
#     span = doc[start:end]  # Matched span
#     sent = span.sent  # Sentence containing matched span
#     # Append mock entity for match in displaCy style to matched_sents
#     # get the match span by ofsetting the start and end of the span with the
#     # start and end of the sentence in the doc
#     match_ents = [{
#         "start": span.start_char - sent.start_char,
#         "end": span.end_char - sent.start_char,
#         "label": "MATCH",
#     }]
#     matched_sents.append({"text": sent.text, "ents": match_ents})

# pattern = [{"LOWER": "facebook"}, {"LEMMA": "be"}, {"POS": "ADV", "OP": "*"},
#            {"POS": "ADJ"}]
# matcher.add("FacebookIs", [pattern], on_match=collect_sents)  # add pattern
# doc = nlp("I'd say that Facebook is evil. – Facebook is pretty cool, right?")
# matches = matcher(doc)

# # Serve visualization of sentences containing match with displaCy
# # set manual=True to make displaCy render straight from a dictionary
# # (if you're not running the code within a Jupyer environment, you can
# # use displacy.serve instead)
# displacy.render(matched_sents, style="ent", manual=True)

## PhraseMatching 

In [ ]:

# nlp = spacy.load("en_core_web_sm")
# matcher = PhraseMatcher(nlp.vocab)
# terms = ["Barack Obama", "Angela Merkel", "Washington, D.C."]
# # Only run nlp.make_doc to speed things up
# patterns = [nlp.make_doc(text) for text in terms]
# matcher.add("TerminologyList", patterns)

# doc = nlp("German Chancellor Angela Merkel and US President Barack Obama "
#           "converse in the Oval Office inside the White House in Washington, D.C.")
# matches = matcher(doc)
# for match_id, start, end in matches:
#     span = doc[start:end]
#     print(span.text)

In [ ]:
# from spacy.lang.en import English
# from spacy.matcher import PhraseMatcher

# nlp = English()
# matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
# patterns = [nlp.make_doc(name) for name in ["Angela Merkel", "Barack Obama"]]
# matcher.add("Names", patterns)

# doc = nlp("angela merkel and us president barack Obama")
# for match_id, start, end in matcher(doc):
#     print("Matched based on lowercase token text:", doc[start:end])

In [ ]:
# matcher: Matcher = Matcher(dadma_nlp.vocab, validate=True)

# for t_index, text in enumerate(texts):
#     text = normalizer.normalize(text)
#     doc = stanza_nlp(text)
#     print(f'Text {t_index + 1}: ...')
        
#     for s_index, sentence in enumerate(doc.sents):
#         print(f'Sentence {s_index + 1}: ...')
#         for token in sentence:
#             print(f"{token.text:10}, {token.pos_:10}, {token.tag_:10}, {token.dep_}")
                
        
#     print("\n\n")

In [ ]:
# from spacy import displacy

# for index, text in enumerate(texts):
#     text = normalizer.normalize(text)
#     try:
        
#         doc = dadma_nlp(text)
        
#         # print(f'sentence {index + 1}: ...')
#         # for token in doc:
#         #     print(f'word: {token.text:12}, pos: {token.pos_:10}, tag: {token.tag_:10}, dep: {token.dep_:15}')
#         #     print("\n")
        
#         sentences = doc._.sentences
        
#         # for sentence in sentences:
#         #     sentence_text = sentence.text
#         #     for token in sentence:
#         #         token_text = token.text
#         #         lemma = token.lemma_ ## this has value only if lem is called
#         #         pos_tag = token.pos_ ## this has value only if pos is called
#         #         dep = token.dep_ ## this has value only if dep is called
#         #         dep_arc = token._.dep_arc ## this has value only if dep is called
#         #         print(token_text, pos_tag, dep, dep_arc)
#         #         if token.pos_ == "AUX":
#         #             token.pos_ = "VERB"
                
#         sent_constituency = doc._.constituency ## this has value only if cons is called
#         sent_chunks = doc._.chunks ## this has value only if cons is called
#         # ners = doc._.ners ## this has value only if ner is called
#         # print(sent_constituency)
#         print(sent_chunks)
        
#         print("\n\n\n")
        
#         displacy.render(doc, style="dep")
        
#     except Exception:
        
#         print(f"ERRR {text}")
    
        

## [Pattern Matching](https://spacy.io/usage/spacy-101#architecture-matchers)

